# Creating digital signature with public key

### 1. Signing transactions to create a digital signature appended with public key

In [70]:
from ecdsa import SigningKey, SECP256k1

sk = SigningKey.generate(curve=SECP256k1)
vk = sk.get_verifying_key()

print("sk: ", sk.to_string())
print("vk: ", vk.to_string())

sk:  b'n\xf6\xc0\xc6E\x1e\xc6\x92Ja\x89\xa2\xe7\x92\xd7or\xa2\xbf[\xc4\xe5\xb1\x9f\xa0\xd4\xcd\xa9u\xf6\x177'
vk:  b'\xd8\xacy\xca\xddX\x1diu\x94\x91\xeb\x81\xed\x10^\x1czv\x8e\xee\xa9\xa0\x87\x7f\xcfZ\r\xd5m3\xcdq\x94\x9f\xeayIp\x18\x90\x9e\x1a\xe7\x15\x14\x11\x85[Q\x98\x11\x83>\xd4\xd0\xe1\x0b[7\x9b\xe7\xbc)'


In [71]:
import json 

txns = {"Jimmy": -50, "Alice": 50}
sig = sk.sign(str.encode(json.dumps(txns)))

print("Sig: ", sig)

# Combine signature with 
sig_vk = (sig + vk.to_string()).hex()
print("\nSig_vk:\n", sig_vk)

Sig:  b'\x94\xa0A\xa5\xa7\x81\x82X\xacV\x16E[\x06\xb6\xc2\x9d\x94\xc3z\x96\xf0p\xe0\xcf\x10\xe69\x88\xa6\xc5^\xbb\x17\x04\xcfN\xd2\n\xaau<\x86/\x95\xbc\x19|H\x14FmTh\xbb\xe8\xe9\x12\x8e1h\xcd\xa7;'

Sig_vk:
 94a041a5a7818258ac5616455b06b6c29d94c37a96f070e0cf10e63988a6c55ebb1704cf4ed20aaa753c862f95bc197c4814466d5468bbe8e9128e3168cda73bd8ac79cadd581d69759491eb81ed105e1c7a768eeea9a0877fcf5a0dd56d33cd71949fea79497018909e1ae7151411855b519811833ed4d0e10b5b379be7bc29


In [72]:
# Receiver: 
# - Receives signature in hexidecimal format 
# - Converts to bytes, seperates 

def parse_signature(hex_str): 
    sig_plus_vk = bytes.fromhex(hex_str)
    return sig_plus_vk[:64], sig_plus_vk[64:]

actual_sig, actual_vk = parse_signature(sig_vk)

print("Sig: ", actual_sig)
print("vk: ", actual_vk)

Sig:  b'\x94\xa0A\xa5\xa7\x81\x82X\xacV\x16E[\x06\xb6\xc2\x9d\x94\xc3z\x96\xf0p\xe0\xcf\x10\xe69\x88\xa6\xc5^\xbb\x17\x04\xcfN\xd2\n\xaau<\x86/\x95\xbc\x19|H\x14FmTh\xbb\xe8\xe9\x12\x8e1h\xcd\xa7;'
vk:  b'\xd8\xacy\xca\xddX\x1diu\x94\x91\xeb\x81\xed\x10^\x1czv\x8e\xee\xa9\xa0\x87\x7f\xcfZ\r\xd5m3\xcdq\x94\x9f\xeayIp\x18\x90\x9e\x1a\xe7\x15\x14\x11\x85[Q\x98\x11\x83>\xd4\xd0\xe1\x0b[7\x9b\xe7\xbc)'


### 2. Function to generate bitcoin address from public key

In [78]:
from Crypto.Hash import SHA256
from Crypto.Hash.RIPEMD import RIPEMD160Hash
import base58

def generate_bitcoin_address(pubkey):
    
    if len(public_key) != 64: 
        raise Exception("Please enter a")

    hash_1 = SHA256.new(pubkey).hexdigest() # Stage 1 
    r160_hash_obj = RIPEMD160Hash() # Stage 2
    r160_hash_obj.update(hash_1)    
    hash_2 = r160_hash_obj.hexdigest()
    hash_3 = "00" + hash_2 # Stage 3 

    hash_4 = SHA256.new(str.encode(hash_3)).digest().hex() # Stage 4
    hash_5 = SHA256.new(str.encode(hash_4)).digest() # Stage 5
    checksum = hash_5[:4].hex() # Stage 6
    
    bit_address = hash_3 + checksum # Stage 7
    return base58.b58encode(bytes.fromhex(bit_address)) # Stage 8

public_addr = generate_bitcoin_address(vk.to_string())

print("Public address: ", public_addr)

Public address:  1LM4EZYkY8KcL1qrdi994NA9qayv1DtC5Q


### 3. Check the hash to confirm verification keys

In [79]:
if public_addr == generate_bitcoin_address(actual_vk): 
    print("The public key is valid!")
else: 
    print("This key is invalid")


The public key is valid!


### 4. Verify the contents of the document

In [90]:
from ecdsa import SigningKey, VerifyingKey, NIST384p

txns = {"Jimmy": -50, "Alice": 50}
unicode_json_txns = str.encode(json.dumps(txns))

vk_obj = VerifyingKey.from_string(actual_vk, curve=SECP256k1)

vk_obj.verify(actual_sig, unicode_json_txns)

True

This is how you make do it!!